In [1]:
%reload_ext autoreload
%autoreload 2
import chess.pgn
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from chess_dataset import ChessDataset, ChessPairDataset
import helper

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [2]:
train_chess_pair_dataset = ChessPairDataset("dataset/npy/")
test_chess_pair_dataset = ChessPairDataset("dataset/npy/", train=False)
train_chess_dataset = ChessDataset("dataset/npy/")
test_chess_dataset = ChessDataset("dataset/npy/", train=False)

In [3]:
len(train_chess_pair_dataset), len(test_chess_pair_dataset)

(32000, 8000)

In [4]:
len(train_chess_dataset), len(test_chess_dataset)

(2739, 686)

In [5]:
chess_trainloader = DataLoader(train_chess_dataset, batch_size=64, shuffle=True)
chess_testloader = DataLoader(test_chess_dataset, batch_size=64, shuffle=False)
chess_pair_trainloader = DataLoader(train_chess_pair_dataset, batch_size=64, shuffle=True)
chess_pair_testloader = DataLoader(test_chess_pair_dataset, batch_size=64, shuffle=True)

In [ ]:
def freeze(layer):
  layer.weight.requires_grad = False
  layer.bias.requires_grad = False

def unfreeze(layer):
  layer.weight.requires_grad = True
  layer.bias.requires_grad = True
  
# Dense encoding layers
en1 = nn.Linear(773, 600)
en2 = nn.Linear(600, 400)
en3 = nn.Linear(400, 200)
en4 = nn.Linear(200, 100)

# Dense decoding layers
de1 = nn.Linear(600, 773)
de2 = nn.Linear(400, 600)
de3 = nn.Linear(200, 400)
de4 = nn.Linear(100, 200)

# Autoencoder train
# 773 - 600 - 773
ae1 = nn.Sequential(en1, nn.ReLU(), 
                    de1, nn.Sigmoid())
helper.train_autoencoder(ae1, chess_trainloader, chess_testloader, 10, 3)

# * means freeze
# 773 *- 600 - 400 - 600 *- 773
freeze(en1)
freeze(de1)
ae2 = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), 
                    de2, nn.ReLU(), de1, nn.Sigmoid())
helper.train_autoencoder(ae2, chess_trainloader, chess_testloader, 10, 3)

# 773 *- 600 *- 400 - 200 - 400 *- 600 *- 773
freeze(en2)
freeze(de2)
ae3 = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), en3, nn.ReLU(), 
                    de3, nn.ReLU(), de2, nn.ReLU(), de1, nn.Sigmoid())
helper.train_autoencoder(ae3, chess_trainloader, chess_testloader, 10, 3)

# 773 *- 600 *- 400 *- 200 - 100 - 200 *- 400 *- 600 *- 773
freeze(en3)
freeze(de3)
ae4 = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), en3, nn.ReLU(), en4, nn.ReLU(), 
                    de4, nn.ReLU(), de3, nn.ReLU(), de2, nn.ReLU(), de1, nn.Sigmoid())
helper.train_autoencoder(ae4, chess_trainloader, chess_testloader, 10, 3)

unfreeze(en1)
unfreeze(en1)
unfreeze(en3)
pos2vec = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), en3, nn.ReLU(), en4, nn.ReLU())

Epoch: 0 | Loss: 0.1984473019838333: 100%|██████████████████████████████████████| 43/43 [00:01<00:00, 34.26it/s]
Val loss: 0.19835510849952698: 100%|████████████████████████████████████████████| 11/11 [00:00<00:00, 45.34it/s]
Epoch: 1 | Loss: 0.09149955213069916: 100%|█████████████████████████████████████| 43/43 [00:01<00:00, 33.66it/s]
Val loss: 0.08792492747306824: 100%|████████████████████████████████████████████| 11/11 [00:00<00:00, 44.47it/s]
Epoch: 2 | Loss: 0.055318351835012436: 100%|████████████████████████████████████| 43/43 [00:01<00:00, 34.55it/s]
Val loss: 0.04623376578092575: 100%|████████████████████████████████████████████| 11/11 [00:00<00:00, 44.49it/s]
Epoch: 3 | Loss: 0.039942268282175064: 100%|████████████████████████████████████| 43/43 [00:01<00:00, 35.75it/s]
Val loss: 0.0352659709751606: 100%|█████████████████████████████████████████████| 11/11 [00:00<00:00, 44.29it/s]
Epoch: 4 | Loss: 0.034827958792448044: 100%|████████████████████████████████████| 43/43 [00:01<0

In [ ]:
class DeepChess(nn.Module):
  def __init__(self):
    super(DeepChess, self).__init__()
    self.pos2vec = pos2vec
#     self.pos2vec1 = nn.Sequential(nn.Linear(773, 600), nn.ReLU(), nn.BatchNorm1d(600), nn.Dropout(0.5),
#                                  nn.Linear(600, 400), nn.ReLU(), nn.BatchNorm1d(400), nn.Dropout(0.5),
#                                  nn.Linear(400, 200), nn.ReLU(), nn.BatchNorm1d(200), nn.Dropout(0.5),
#                                  nn.Linear(200, 100), nn.ReLU(), nn.BatchNorm1d(100), nn.Dropout(0.5))
    
#     self.pos2vec2 = nn.Sequential(nn.Linear(773, 600), nn.ReLU(), nn.BatchNorm1d(600), nn.Dropout(0.5),
#                                  nn.Linear(600, 400), nn.ReLU(), nn.BatchNorm1d(400), nn.Dropout(0.5),
#                                  nn.Linear(400, 200), nn.ReLU(), nn.BatchNorm1d(200), nn.Dropout(0.5),
#                                  nn.Linear(200, 100), nn.ReLU(), nn.BatchNorm1d(100), nn.Dropout(0.5))
    
    self.base = nn.Sequential(nn.Linear(200, 400), nn.ReLU(), nn.BatchNorm1d(400), nn.Dropout(0.5),
                              nn.Linear(400, 200), nn.ReLU(), nn.BatchNorm1d(200), nn.Dropout(0.5),
                              nn.Linear(200, 100), nn.ReLU(), nn.BatchNorm1d(100), nn.Dropout(0.5),
                              nn.Linear(100, 2), nn.Softmax(dim=1))
  
  def forward(self, input1, input2):
    first_board = self.pos2vec(input1)
    second_board = self.pos2vec(input2)
    concat = torch.cat((first_board, second_board), 1)
    return self.base(concat)

In [ ]:
dp = DeepChess()
dp

In [ ]:
helper.train_supervise(model=dp, train_data=chess_pair_trainloader, val_data=chess_pair_testloader, epochs=10, patient=2)

In [ ]:
i = next(iter(chess_pair_testloader))
z = i[2]
zz = dp(i[0], i[1])
print(torch.argmax(z, 1))
#print(zz)
print(torch.argmax(zz, 1))
# for a in zip(z, zz):
#   print(a)      
print(sum(1 for x in (torch.argmax(z, 1) == torch.argmax(zz, 1)) if x), "/", i[0].shape[0])

In [ ]:
torch.save(dp.state_dict(), "model/save/deepchess.pth")
torch.save(pos2vec.state_dict(), "model/save/pos2vec.pth")